In [87]:
from google.cloud.sql.connector import connector
import sqlalchemy
import pymysql
import datetime
import pprint
import random

**Create Config Database**

In [39]:
db_config = {
        # [START cloud_sql_mysql_sqlalchemy_limit]
        # Pool size is the maximum number of permanent connections to keep.
        "pool_size": 5,
        # Temporarily exceeds the set pool_size if no connections are available.
        "max_overflow": 2,
        # The total number of concurrent connections for your application will be
        # a total of pool_size and max_overflow.
        # [END cloud_sql_mysql_sqlalchemy_limit]

        # [START cloud_sql_mysql_sqlalchemy_backoff]
        # SQLAlchemy automatically uses delays between failed connection attempts,
        # but provides no arguments for configuration.
        # [END cloud_sql_mysql_sqlalchemy_backoff]

        # [START cloud_sql_mysql_sqlalchemy_timeout]
        # 'pool_timeout' is the maximum number of seconds to wait when retrieving a
        # new connection from the pool. After the specified amount of time, an
        # exception will be thrown.
        "pool_timeout": 30,  # 30 seconds
        # [END cloud_sql_mysql_sqlalchemy_timeout]

        # [START cloud_sql_mysql_sqlalchemy_lifetime]
        # 'pool_recycle' is the maximum number of seconds a connection can persist.
        # Connections that live longer than the specified amount of time will be
        # reestablished
        "pool_recycle": 1800,  # 30 minutes
        # [END cloud_sql_mysql_sqlalchemy_lifetime]
}

db_user = "root"
db_pass = "EuJaEstouComSaudadeDoKanji"
db_name = "cloud-sql-workshop"

**Connection Database** 

In [118]:
def create_connection(ip_address):
    # Extract port from db_host if present,
    # otherwise use DB_PORT environment variable.
    host_args = ip_address.split(":")
    if len(host_args) == 1:
        db_hostname = db_host
        db_port = 3306
    elif len(host_args) == 2:
        db_hostname, db_port = host_args[0], int(host_args[1])

    return sqlalchemy.create_engine(
        # Equivalent URL:
        # mysql+pymysql://<db_user>:<db_pass>@<db_host>:<db_port>/<db_name>
        sqlalchemy.engine.url.URL.create(
            drivername="mysql+pymysql",
            username=db_user, 
            password=db_pass, 
            host=db_hostname, 
            port=db_port,  
            database=db_name,
        ),
        **db_config
    )

master = create_connection("10.77.0.2")
replica1 = create_connection("10.77.0.6")
replica2 = create_connection("10.77.1.3")

**Create TABLE**

In [49]:
with master.connect() as conn:
        conn.execute(
            "CREATE TABLE IF NOT EXISTS votes "
            "( vote_id SERIAL NOT NULL, time_cast timestamp NOT NULL, "
            "candidate CHAR(6) NOT NULL, PRIMARY KEY (vote_id) );"
        )


**INSERT** 

In [102]:
# Get the team and time the vote was cast.
def insert_data(team, pool):
    time_cast = datetime.datetime.now(tz=datetime.timezone.utc)

    stmt = sqlalchemy.text(
            "INSERT INTO votes (time_cast, candidate)" " VALUES (:time_cast, :candidate)"
    )

    with pool.connect() as conn:
            conn.execute(stmt, time_cast=time_cast, candidate=team)

**GET DATA**

In [65]:
def get_data(pool):
    votes = []
    with pool.connect() as conn:
        # Execute the query and fetch all results
        recent_votes = conn.execute(
            "SELECT candidate, time_cast FROM votes " "ORDER BY time_cast DESC LIMIT 5"
        ).fetchall()
        # Convert the results into a list of dicts representing votes
        for row in recent_votes:
            votes.append({"candidate": row[0], "time_cast": row[1]})

        stmt = sqlalchemy.text(
            "SELECT COUNT(vote_id) FROM votes WHERE candidate=:candidate"
        )
        # Count number of votes for tabs
        team_a_result = conn.execute(stmt, candidate="A").fetchone()
        team_a_count = team_a_result[0]
        # Count number of votes for spaces
        team_b_result = conn.execute(stmt, candidate="B").fetchone()
        team_b_count = team_b_result[0]
    return {
        'recent_votes': votes,
        'team A': team_a_count,
        'team B': team_b_count,
    }

In [127]:
for x in range(500):
    try:
        insert_data("A" if bool(random.getrandbits(1)) else "B", master)
    except Exception:
        print("Error insert Master")
        master = create_connection("10.77.0.2")

try:
    pprint.pprint(get_data(master))
except Exception:
    print("Error Read Master")
    master = create_connection("10.77.0.2")
    
try:
    pprint.pprint(get_data(replica1))
except Exception:
    print("Error Read Replica 1")
    replica1 = create_connection("10.77.0.6")
    
try:
    pprint.pprint(get_data(replica2))
except Exception:
    print("Error Read Replica 2")
    replica2 = create_connection("10.77.1.3")



{'recent_votes': [{'candidate': 'A',
                   'time_cast': datetime.datetime(2021, 11, 17, 5, 52, 10)},
                  {'candidate': 'A',
                   'time_cast': datetime.datetime(2021, 11, 17, 5, 52, 10)},
                  {'candidate': 'A',
                   'time_cast': datetime.datetime(2021, 11, 17, 5, 52, 10)},
                  {'candidate': 'B',
                   'time_cast': datetime.datetime(2021, 11, 17, 5, 52, 10)},
                  {'candidate': 'A',
                   'time_cast': datetime.datetime(2021, 11, 17, 5, 52, 10)}],
 'team A': 2470,
 'team B': 2557}
{'recent_votes': [{'candidate': 'A',
                   'time_cast': datetime.datetime(2021, 11, 17, 5, 52, 10)},
                  {'candidate': 'A',
                   'time_cast': datetime.datetime(2021, 11, 17, 5, 52, 10)},
                  {'candidate': 'A',
                   'time_cast': datetime.datetime(2021, 11, 17, 5, 52, 10)},
                  {'candidate': 'B',
                